# Titanic - Machine Learning from Disaster



## Preparando Setup do código

In [1]:
# Bibliotecas necessarias
import numpy as np
import pandas as pd

# Importando modelos usados
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Objetos necessarios para validação 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold

## Lendo e carregando dados

In [2]:
train_path = './data/train.csv'
test_path = './data/test.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Engenharia de Features

### Função usada para criar coluna relacionada ao sexo do individuo

In [4]:
def format_sex_column(val):
    if val == 'male': 
        return 1
    elif val == 'female': 
        return 0
    else:
        return -1

### Criando e formatando colunas categoricas para numericas 

- __Sex_code__: Representação numerica do sexo.

Aqui também são excluidos os valores __Nulos__ presentes na coluna que representa a Idade dos individuos.


In [5]:
train_data['Sex_code'] = train_data['Sex'].map(format_sex_column)
train_data['Age'].fillna(-1,inplace = True)

test_data['Sex_code'] = test_data['Sex'].map(format_sex_column)
test_data['Age'].fillna(-1,inplace = True)

## Treinamento & Validação
### selecionando de features e target para treinamento e avaliação do nosso modelo

As features escolhidas para o treinamento do modelo foram:
   - Sex_code: Variavel definida com base no Sexo do individuo.
   - Age: Idade do individuo.

In [6]:
features = ['Sex_code', 'Age']

X = train_data[features]
y = train_data['Survived']

In [7]:
predicts = [] #array que guarda todas as predioções feitas
kfold = RepeatedKFold(n_splits = 2, n_repeats = 20, random_state = 42069)

for train_index, test_index in kfold.split(X):
    # Dados de treino e test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Modelo sendo criado, treinado e predizendo resultados
    decisionTree = DecisionTreeClassifier()
    decisionTree.fit(X_train,y_train)
    decisionTree_result = decisionTree.predict(X_test)
    
    # Salvando resultados para analise
    predicts.append(accuracy_score(y_test,decisionTree_result))

np.mean(predicts)

0.7499972288003225

## Detalhes sobre a tentativa

- __Modelo utilizado__: Decision Tree Classifier
- __Features usadas para treino__: "Age" e "Sex"
- __Target__: "Survived"

Utilizando apenas as colunas mais intuitivas e o modelo de arvore de decisão, segundo o __Kaggle__ obtivemos a pontuação de __0.69856__.

## Otimização a partir de criação de novas colunas

### Adicionando outra função para remodelagem de coluna 

A função em questão nos auxilia na criação de uma nova coluna chamada __Embarked_code__. A função transforma os valores categoricos da coluna '__Embarqued__', de forma que o nosso modelo possa aproveitar suas informações.

In [8]:
def format_embarked_column(val):
    if val == 'C':
        return 1
    elif val == 'Q':
        return 2
    elif val == 'S':
        return 3
    else:
        return -1

### Criação colunas que serão usadas com features de previsão.

- __Embarked_code__: Representação numerica do porto no qual a pessoa embarcou.
- __Family_len__: Numero de familiares abordo.

As colunas __is_Col__, __is_Major__, __is_Master__, __is_Mr__, __is_Miss__ e __is_Mrs__ são todas relacionadas ao titulo do individuo.


In [9]:
train_data['Embarked_code'] = train_data['Embarked'].map(format_embarked_column)
train_data['Family_len'] = train_data['SibSp'] + train_data['Parch']
train_data['is_Col'] = train_data['Name'].str.contains('Col')
train_data['is_Major'] = train_data['Name'].str.contains('Major')
train_data['is_Master'] = train_data['Name'].str.contains('Master')
train_data['is_Mr'] = train_data['Name'].str.contains('Mr')
train_data['is_Miss'] = train_data['Name'].str.contains('Miss')
train_data['is_Mrs'] = train_data['Name'].str.contains('Mrs')

test_data['Embarked_code'] = test_data['Embarked'].map(format_embarked_column)
test_data['Family_len'] = test_data['SibSp'] + test_data['Parch']
test_data['is_Col'] = test_data['Name'].str.contains('Col')
test_data['is_Major'] = test_data['Name'].str.contains('Major')
test_data['is_Master'] = test_data['Name'].str.contains('Master')
test_data['is_Mr'] = test_data['Name'].str.contains('Mr')
test_data['is_Miss'] = test_data['Name'].str.contains('Miss')
test_data['is_Mrs'] = test_data['Name'].str.contains('Mrs')

## Treinamento & Validação
### selecionando de features e target para treinamento e avaliação do nosso modelo

As features escolhidas para o treinamento do modelo foram:
   - Sex_code: Variavel definida com base no Sexo do individuo.
   - Pclass: Variavel que define com a classe cujo individuo está viajando.
   - Age: Idade do individuo.
   - Family_len: tamanho da familia do individuo.
   - Embarked_code: Codigo que se refere ao local de embarque do individuo.

As colunas __is_Col__, __is_Major__, __is_Master__, __is_Mr__, __is_Miss__ e __is_Mrs__ são todas relacionadas ao titulo do individuo.

In [10]:
features = ['Sex_code', 'Pclass','Age','SibSp',
            'Parch', 'Family_len', 'Embarked_code',
            'is_Col', 'is_Major', 'is_Master', 'is_Mr', 
            'is_Miss', 'is_Mrs']

X = train_data[features]
y = train_data['Survived']

## Validação

O metodo usado na avaliação do modelo é o conhecido como "__Cross-Validation__".

In [11]:
predicts = [] #array que guarda todas as predioções feitas
kfold = RepeatedKFold(n_splits = 2, n_repeats = 20, random_state = 56732)

for train_index, test_index in kfold.split(X):
    # Dados de treino e test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Modelo sendo criado, treinado e predizendo resultados
    logist = LogisticRegression(solver = 'liblinear', C = 1, penalty = 'l2', multi_class = 'ovr')
    logist.fit(X_train,y_train)
    logist_result = logist.predict(X_test)
    
    # Salvando resultados para analise
    predicts.append(accuracy_score(y_test,logist_result))
    
np.mean(predicts)

0.8170077341663727

## Detalhes sobre a tentativa

- __Modelo utilizado__: Logistic Regression
- __Features usadas para treino__: "Age", "Sex", "Pclass", "Age", "SibSp", "Parch", "Family_len", "Embarked_code", "is_Col", "is_Major", "is_Master", "is_Mr", "is_Miss", "is_Mrs"
- __Target__: "Survived"

Após uma mudança de modelo, adição e criação de algumas outras colunas, percebemos um notorio aumento na pontuação de validação. Segundo o __Kaggle__ obtivemos a pontuação de __0.77990__.

### Treinamento

Treinando o modelo novamente, dessa vez utilizando todo o conjunto de dados de __treino__.

In [12]:
logist = LogisticRegression(solver = 'liblinear', C = 1, penalty = 'l2', multi_class = 'ovr')
logist.fit(X,y)

LogisticRegression(C=1, multi_class='ovr', solver='liblinear')

### Prevendo dados finais

In [13]:
test_data_fill = test_data[features]
resultado_final = logist.predict(test_data_fill)

## Criando arquivo de submissão

In [14]:
pred = pd.Series(resultado_final, index= test_data['PassengerId'], name = 'Survived')

In [15]:
pred.to_csv('previsao_regressao_logistica.csv', header = True)

# Observações finais

- __Modelo utilizado__:
    Podemos buscar por um modelo que se encaixe ainda mais com o problema em si. Algum modelo que seja mais sensível a mudança de hyper parâmetros e a partir disso seja possível Tunar o modelo.

- __Tratamento de dados__:
    Um próximo passo a ser tomado, é em relação a um estudo de resultados, dentro do nosso conjunto de validação. Para que a partir disso, nós possamos ter ideia de melhores features a serem usadas.